In [33]:
!git clone https://github.com/pthengtr/kcw-analytics.git

fatal: destination path 'kcw-analytics' already exists and is not an empty directory.


In [34]:
!cd /content/kcw-analytics && git pull origin main

From https://github.com/pthengtr/kcw-analytics
 * branch            main       -> FETCH_HEAD
Already up to date.


In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import os
import pandas as pd

folder = "/content/drive/MyDrive/kcw_analytics/01_raw"

data = {}

for file in os.listdir(folder):
    if file.endswith(".csv"):
        path = os.path.join(folder, file)
        data[file] = pd.read_csv(
            path,
            dtype={
              "BCODE": "string",
              "ITEMNO": "string",
              "BILLNO": "string",
            },
            encoding="utf-8-sig",
            low_memory=False   # stops chunk guessing
        )
        print(f"Loaded: {file} -> {data[file].shape}")



Loaded: raw_inventory_hq_2024.csv -> (4983, 8)
Loaded: raw_syp_pimas_purchase_bills.csv -> (2881, 49)
Loaded: raw_syp_pidet_purchase_lines.csv -> (26919, 41)
Loaded: raw_syp_sidet_sales_lines.csv -> (35191, 38)
Loaded: raw_syp_simas_sales_bills.csv -> (11923, 49)
Loaded: raw_hq_pimas_purchase_bills.csv -> (83416, 49)
Loaded: raw_hq_pidet_purchase_lines.csv -> (248835, 41)
Loaded: raw_hq_icmas_products.csv -> (114874, 94)
Loaded: raw_hq_sidet_sales_lines.csv -> (1200038, 38)
Loaded: raw_hq_simas_sales_bills.csv -> (486118, 49)


In [37]:
hq_sales_lines = data['raw_hq_sidet_sales_lines.csv'].copy()
syp_sales_lines = data['raw_syp_sidet_sales_lines.csv'].copy()
purchase_lines = data['raw_hq_pidet_purchase_lines.csv'].copy()

In [38]:
import sys
import importlib

# ensure repo is on path
repo_path = "/content/kcw-analytics"
if repo_path not in sys.path:
    sys.path.append(repo_path)

# import the module (NOT individual functions)
import src.kcw.supabase_utils as supabase_utils

# reload to pick up latest .py changes
importlib.reload(supabase_utils)

add_sales_quality_flags = supabase_utils.add_sales_quality_flags
enrich_sales_with_last_purchase_cost = supabase_utils.enrich_sales_with_last_purchase_cost
qc_unknown = supabase_utils.qc_unknown
refill_last_cost_from_icmas = supabase_utils.refill_last_cost_from_icmas
build_all_dims = supabase_utils.build_all_dims

In [39]:
hq_sales_lines.columns

Index(['ID', 'JOURMODE', 'JOURTYPE', 'JOURDATE', 'BILLTYPE', 'BILLDATE',
       'BILLNO', 'LINE', 'ITEMNO', 'BCODE', 'PCODE', 'MCODE', 'DETAIL',
       'WHNUMBER', 'LOCATION1', 'STATUS', 'SERIAL', 'TAXIC', 'EXMPT', 'ISVAT',
       'QTY', 'UI', 'MTP', 'PRICE', 'XPRICE', 'DISCNT1', 'DISCNT2', 'DISCNT3',
       'DISCNT4', 'DED', 'VAT', 'AMOUNT', 'CHGAMT', 'ACCTNO', 'PAID',
       'ACCT_NO', 'DONE', 'CANCELED'],
      dtype='object')

In [40]:
syp_sales_flagged = add_sales_quality_flags(syp_sales_lines)

# QC summary (no deletion)
total = len(syp_sales_flagged)
invalid = (~syp_sales_flagged["IS_VALID"]).sum()
print(f"Invalid: {invalid:,}/{total:,} ({invalid/total*100:.2f}%)")

print(
    syp_sales_flagged["INVALID_REASON"]
    .fillna("OK")
    .value_counts()
    .head(20)
)

# For analytics (optional): just filter in pandas
syp_sales_valid_only = syp_sales_flagged[syp_sales_flagged["IS_VALID"]].copy()


Invalid: 13/35,191 (0.04%)
INVALID_REASON
OK            35178
BAD_BCODE         7
BAD_AMOUNT        6
Name: count, dtype: Int64


In [41]:
hq_sales_flagged = add_sales_quality_flags(hq_sales_lines)

# QC summary (no deletion)
total = len(hq_sales_flagged)
invalid = (~hq_sales_flagged["IS_VALID"]).sum()
print(f"Invalid: {invalid:,}/{total:,} ({invalid/total*100:.2f}%)")

print(
    hq_sales_flagged["INVALID_REASON"]
    .fillna("OK")
    .value_counts()
    .head(20)
)

# For analytics (optional): just filter in pandas
syp_sales_valid_only = hq_sales_flagged[hq_sales_flagged["IS_VALID"]].copy()

Invalid: 31,343/1,200,038 (2.61%)
INVALID_REASON
OK                                1168695
BAD_BCODE                           28571
CANCELED                             1814
BAD_AMOUNT                            405
BAD_BCODE|BAD_PRICE                   280
BAD_PRICE                             132
BAD_BCODE|BAD_PRICE|BAD_AMOUNT         94
BAD_PRICE|BAD_AMOUNT                   29
BAD_BCODE|CANCELED                     17
BAD_AMOUNT|CANCELED                     1
Name: count, dtype: Int64


In [42]:
hq_sales_enriched = enrich_sales_with_last_purchase_cost(
    hq_sales_flagged,
    purchase_lines,
)

qc_unknown(hq_sales_enriched, "before refill")

hq_sales_enriched = refill_last_cost_from_icmas(
    data,
    hq_sales_enriched,
    last_cost_col="LAST_PURCHASE_COST",
)

qc_unknown(hq_sales_enriched, "after refill")



[before refill] UNKNOWN: 69,161 / 1,200,038 (5.76%)
[after refill] UNKNOWN: 69,161 / 1,200,038 (5.76%)


In [43]:
syp_sales_enriched = enrich_sales_with_last_purchase_cost(
    syp_sales_flagged,
    purchase_lines,
)

qc_unknown(syp_sales_enriched, "before refill")

syp_sales_enriched = refill_last_cost_from_icmas(
    data,
    syp_sales_enriched,
    last_cost_col="LAST_PURCHASE_COST",
)

qc_unknown(syp_sales_enriched, "after refill")

[before refill] UNKNOWN: 520 / 35,191 (1.48%)
[after refill] UNKNOWN: 520 / 35,191 (1.48%)


In [44]:
hq_sales_enriched["BRANCH"] = "HQ"
syp_sales_enriched["BRANCH"] = "SYP"

In [45]:
sales_all = pd.concat([hq_sales_enriched, syp_sales_enriched], ignore_index=True)

sales_all["BRANCH"] = sales_all["BRANCH"].astype("string")
sales_all["LAST_PURCHASE_COST"] = pd.to_numeric(sales_all["LAST_PURCHASE_COST"], errors="coerce")
sales_all["BILLDATE"] = pd.to_datetime(sales_all["BILLDATE"], errors="coerce")

sales_all["BRANCH_BILLNO"] = sales_all["BRANCH"] + "-" + sales_all["BILLNO"].astype(str)


In [46]:
sales_all["BILLTYPE_STD"] = (
    sales_all["BILLNO"]
    .astype("string")
    .str.upper()
    .str.replace(r"^3", "", regex=True)   # remove leading 3 if exists
    .str.extract(r"^(TFV|TAD|TAR|TR|TD|TF|CN|DN)", expand=False)
    .fillna("UNKNOWN")
)

In [47]:
KEEP_COLS = [
    'BILLDATE', 'BILLTYPE', 'JOURMODE',
    'BILLNO', 'BCODE', 'DETAIL',
    'STATUS', 'ISVAT',
    'QTY', 'UI', 'MTP', 'PRICE', 'XPRICE', 'DISCNT1', 'DISCNT2', 'DISCNT3',
    'DISCNT4', 'DED', 'VAT', 'AMOUNT', 'ACCTNO', 'PAID',
    'ACCT_NO', 'DONE', 'CANCELED',
    'PRICE_NUM', 'AMOUNT_NUM', 'IS_VALID', 'INVALID_REASON', 'ROW_ID',
    'LAST_PURCHASE_DATE', 'LAST_PURCHASE_COST', 'COST_STATUS',
    'BRANCH', 'BRANCH_BILLNO', 'BILLTYPE_STD'
]

# keep only columns that actually exist (prevents KeyError)
sales_all = sales_all[[c for c in KEEP_COLS if c in sales_all.columns]].copy()

In [48]:
sales_all

,BILLDATE,BILLTYPE,JOURMODE,BILLNO,BCODE,DETAIL,STATUS,ISVAT,QTY,UI,...,AMOUNT_NUM,IS_VALID,INVALID_REASON,ROW_ID,LAST_PURCHASE_DATE,LAST_PURCHASE_COST,COST_STATUS,BRANCH,BRANCH_BILLNO,BILLTYPE_STD
0,2015-06-20,1.0,2,KC1506-0006,<NA>,สลักปีกนก,1.0,N,1.0,ตัว,...,270.0,False,BAD_BCODE,0,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
1,2015-06-20,1.0,2,KC1506-0006,<NA>,ลูกหมากปีกนกแท้ D-MAX,1.0,N,1.0,ตัว,...,750.0,False,BAD_BCODE,1,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
2,2015-06-20,1.0,2,KC1506-0006,<NA>,แผ่นผ้าทราย,1.0,N,2.0,แผ่น,...,30.0,False,BAD_BCODE,2,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
3,2015-06-20,1.0,2,KC1506-0006,<NA>,ยางกันโครง,1.0,N,1.0,ตัว,...,20.0,False,BAD_BCODE,3,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
4,2015-06-20,1.0,2,KC1506-0006,<NA>,ยางรองคอยสปริง,1.0,N,2.0,ตัว,...,140.0,False,BAD_BCODE,4,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235224,2026-02-18,1.0,0,3TAR6902-369,33010046,หัวสาย+ปลอก ISR 0606 JS (21MM.),1.0,Y,1.0,ชุด,...,90.0,True,<NA>,35186,2026-02-03,35.0000,OK,SYP,SYP-3TAR6902-369,TAR
1235225,2026-02-18,1.0,0,3TAR6902-370,33010158,"หัวสาย งอ+ปลอก ISR GJS 06-06 90""",1.0,Y,1.0,ชุด,...,140.0,True,<NA>,35187,2025-12-01,61.0000,OK,SYP,SYP-3TAR6902-370,TAR
1235226,2026-02-18,1.0,0,3TAR6902-370,13029051,สายน้ำมัน 2.5 หุน ผ้าใบ 3ชั้น,1.0,Y,2.0,ม.,...,160.0,True,<NA>,35188,2025-12-25,27.5000,OK,SYP,SYP-3TAR6902-370,TAR
1235227,2026-02-18,1.0,0,3TAR6902-370,25035025,ยางโอริง 3.5x25m,1.0,Y,2.0,ตัว,...,30.0,True,<NA>,35189,2025-06-17,3.9400,OK,SYP,SYP-3TAR6902-370,TAR


In [49]:
dims = build_all_dims(sales_all)
{k: v.shape for k, v in dims.items()}

{'dim_date': (3899, 8),
 'dim_product': (34114, 6),
 'dim_category': (37, 2),
 'dim_account': (2956, 4),
 'dim_branch': (2, 3),
 'dim_billtype': (9, 2)}

In [50]:
import shutil
from pathlib import Path

out_dir = Path("/content/drive/MyDrive/kcw_analytics/03_curated")

if out_dir.exists():
    shutil.rmtree(out_dir)


In [51]:
import os
os.makedirs(out_dir, exist_ok=True)

for name, df in dims.items():
    df.to_csv(f"{out_dir}/{name}.csv", index=False, encoding="utf-8-sig")


In [52]:
sales_all.to_csv(
    f"{out_dir}/fact_sales_all.csv",
    index=False,
    encoding="utf-8-sig"   # important for Thai + Excel
)

**DEBUG**

In [53]:
sales_all

,BILLDATE,BILLTYPE,JOURMODE,BILLNO,BCODE,DETAIL,STATUS,ISVAT,QTY,UI,...,AMOUNT_NUM,IS_VALID,INVALID_REASON,ROW_ID,LAST_PURCHASE_DATE,LAST_PURCHASE_COST,COST_STATUS,BRANCH,BRANCH_BILLNO,BILLTYPE_STD
0,2015-06-20,1.0,2,KC1506-0006,<NA>,สลักปีกนก,1.0,N,1.0,ตัว,...,270.0,False,BAD_BCODE,0,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
1,2015-06-20,1.0,2,KC1506-0006,<NA>,ลูกหมากปีกนกแท้ D-MAX,1.0,N,1.0,ตัว,...,750.0,False,BAD_BCODE,1,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
2,2015-06-20,1.0,2,KC1506-0006,<NA>,แผ่นผ้าทราย,1.0,N,2.0,แผ่น,...,30.0,False,BAD_BCODE,2,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
3,2015-06-20,1.0,2,KC1506-0006,<NA>,ยางกันโครง,1.0,N,1.0,ตัว,...,20.0,False,BAD_BCODE,3,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
4,2015-06-20,1.0,2,KC1506-0006,<NA>,ยางรองคอยสปริง,1.0,N,2.0,ตัว,...,140.0,False,BAD_BCODE,4,2015-06-19,NaN,OK,HQ,HQ-KC1506-0006,UNKNOWN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235224,2026-02-18,1.0,0,3TAR6902-369,33010046,หัวสาย+ปลอก ISR 0606 JS (21MM.),1.0,Y,1.0,ชุด,...,90.0,True,<NA>,35186,2026-02-03,35.0000,OK,SYP,SYP-3TAR6902-369,TAR
1235225,2026-02-18,1.0,0,3TAR6902-370,33010158,"หัวสาย งอ+ปลอก ISR GJS 06-06 90""",1.0,Y,1.0,ชุด,...,140.0,True,<NA>,35187,2025-12-01,61.0000,OK,SYP,SYP-3TAR6902-370,TAR
1235226,2026-02-18,1.0,0,3TAR6902-370,13029051,สายน้ำมัน 2.5 หุน ผ้าใบ 3ชั้น,1.0,Y,2.0,ม.,...,160.0,True,<NA>,35188,2025-12-25,27.5000,OK,SYP,SYP-3TAR6902-370,TAR
1235227,2026-02-18,1.0,0,3TAR6902-370,25035025,ยางโอริง 3.5x25m,1.0,Y,2.0,ตัว,...,30.0,True,<NA>,35189,2025-06-17,3.9400,OK,SYP,SYP-3TAR6902-370,TAR


In [54]:
df_unknown = syp_sales_enriched[syp_sales_enriched["LAST_PURCHASE_COST"].isna()]
df_unknown


,ID,JOURMODE,JOURTYPE,JOURDATE,BILLTYPE,BILLDATE,BILLNO,LINE,ITEMNO,BCODE,...,CANCELED,PRICE_NUM,AMOUNT_NUM,IS_VALID,INVALID_REASON,ROW_ID,LAST_PURCHASE_DATE,LAST_PURCHASE_COST,COST_STATUS,BRANCH
50,20050,2,SJ,2025-06-25,1,2025-06-25,3K68-0000032,10,<NA>,70010011,...,N,400.0,-400.0,True,<NA>,50,NaT,NaN,UNKNOWN,SYP
198,70107,2,SJ,2025-07-01,1,2025-07-01,3K68-0000122,10,<NA>,13010000,...,N,1000.0,1000.0,True,<NA>,198,NaT,NaN,UNKNOWN,SYP
221,70133,2,SJ,2025-07-01,1,2025-07-01,3K68-0000135,50,<NA>,13010000,...,N,1000.0,-1000.0,True,<NA>,221,NaT,NaN,UNKNOWN,SYP
239,70155,2,SJ,2025-07-01,1,2025-07-01,3K68-0000143,20,<NA>,70010011,...,N,400.0,-400.0,True,<NA>,239,NaT,NaN,UNKNOWN,SYP
452,100340,2,SJ,2025-07-04,1,2025-07-04,3K68-0000193,10,<NA>,70010011,...,N,400.0,-400.0,True,<NA>,452,NaT,NaN,UNKNOWN,SYP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34556,1987693,2,SJ,2026-02-14,1,2026-02-14,33K69-0001181,10,<NA>,13051935,...,N,10.0,10.0,True,<NA>,34556,NaT,NaN,UNKNOWN,SYP
34700,1997669,2,SJ,2026-02-16,1,2026-02-16,33K69-0001204,20,<NA>,70010400,...,N,400.0,-400.0,True,<NA>,34700,NaT,NaN,UNKNOWN,SYP
34757,2007606,2,SJ,2026-02-17,1,2026-02-17,33K69-0001210,10,<NA>,70010400,...,N,400.0,-400.0,True,<NA>,34757,NaT,NaN,UNKNOWN,SYP
34846,2007708,2,SJ,2026-02-17,1,2026-02-17,32K69-0001212,10,<NA>,70010700,...,N,700.0,-700.0,True,<NA>,34846,NaT,NaN,UNKNOWN,SYP


In [55]:
pidet = data["raw_hq_pidet_purchase_lines.csv"].copy()
pidet_filtered = pidet[pidet["BCODE"] == "13010000"].copy()

pidet_filtered[["BCODE", 'BILLDATE', "QTY", "MTP", "PRICE", "AMOUNT"]]

,BCODE,BILLDATE,QTY,MTP,PRICE,AMOUNT


In [56]:
df_tfv_syp = sales_all[
    (sales_all["BILLTYPE_STD"] == "TAR") &
    (sales_all["BRANCH"] == "SYP")
]

df_tfv_syp

,BILLDATE,BILLTYPE,JOURMODE,BILLNO,BCODE,DETAIL,STATUS,ISVAT,QTY,UI,...,AMOUNT_NUM,IS_VALID,INVALID_REASON,ROW_ID,LAST_PURCHASE_DATE,LAST_PURCHASE_COST,COST_STATUS,BRANCH,BRANCH_BILLNO,BILLTYPE_STD
1200048,2025-06-23,1.0,0,3TAR6806-001,15018750,ลูกปืน 30-72-19 6306 2RS มีฝายาง,1.0,Y,1.0,ตับ,...,170.0,True,<NA>,10,2025-05-15,109.35000,OK,SYP,SYP-3TAR6806-001,TAR
1200049,2025-06-23,1.0,0,3TAR6806-001,15013500,ลูกปืน (ล้อหน้า L2600) 6205 2RSCM(ล้อหน้า L,1.0,Y,1.0,ตับ,...,85.0,True,<NA>,11,2025-06-13,51.40000,OK,SYP,SYP-3TAR6806-001,TAR
1200050,2025-06-23,1.0,0,3TAR6806-001,13022630,"สายพาน 52"" 12.5x1350",1.0,Y,1.0,เส้น,...,190.0,True,<NA>,12,2025-05-05,116.91250,OK,SYP,SYP-3TAR6806-001,TAR
1200051,2025-06-23,1.0,0,3TAR6806-001,14050200,สีสเปรย์ #36(No.300) บอร์นเงิน,1.0,Y,1.0,ก.ป.,...,45.0,True,<NA>,13,2025-06-10,30.37375,OK,SYP,SYP-3TAR6806-001,TAR
1200052,2025-06-23,1.0,0,3TAR6806-001,22050259,น้ำมันเบรค Brembo 0.5 L,1.0,Y,1.0,กป.,...,110.0,True,<NA>,14,2024-11-16,81.00000,OK,SYP,SYP-3TAR6806-001,TAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235224,2026-02-18,1.0,0,3TAR6902-369,33010046,หัวสาย+ปลอก ISR 0606 JS (21MM.),1.0,Y,1.0,ชุด,...,90.0,True,<NA>,35186,2026-02-03,35.00000,OK,SYP,SYP-3TAR6902-369,TAR
1235225,2026-02-18,1.0,0,3TAR6902-370,33010158,"หัวสาย งอ+ปลอก ISR GJS 06-06 90""",1.0,Y,1.0,ชุด,...,140.0,True,<NA>,35187,2025-12-01,61.00000,OK,SYP,SYP-3TAR6902-370,TAR
1235226,2026-02-18,1.0,0,3TAR6902-370,13029051,สายน้ำมัน 2.5 หุน ผ้าใบ 3ชั้น,1.0,Y,2.0,ม.,...,160.0,True,<NA>,35188,2025-12-25,27.50000,OK,SYP,SYP-3TAR6902-370,TAR
1235227,2026-02-18,1.0,0,3TAR6902-370,25035025,ยางโอริง 3.5x25m,1.0,Y,2.0,ตัว,...,30.0,True,<NA>,35189,2025-06-17,3.94000,OK,SYP,SYP-3TAR6902-370,TAR


In [57]:
sales_filtered = sales_all[
    (sales_all["JOURMODE"] == 0) &
    (~sales_all["BILLNO"].astype("string").str.contains("TAR", na=False)) &
    (sales_all["BILLDATE"].dt.year == 2025)
].copy()



sales_filtered

,BILLDATE,BILLTYPE,JOURMODE,BILLNO,BCODE,DETAIL,STATUS,ISVAT,QTY,UI,...,AMOUNT_NUM,IS_VALID,INVALID_REASON,ROW_ID,LAST_PURCHASE_DATE,LAST_PURCHASE_COST,COST_STATUS,BRANCH,BRANCH_BILLNO,BILLTYPE_STD
985753,2025-01-01,1.0,0,TAD6801-001,22023880,จารบีเทรนทอง HT 5Kg TRANE,1.0,Y,1.0,กป.,...,700.0,True,<NA>,985753,2024-11-14,595.794750,OK,HQ,HQ-TAD6801-001,TAD
985754,2025-01-01,1.0,0,TAD6801-001,21040180,แบตเตอรี่ FB N100 FB,1.0,Y,2.0,ลูก,...,7100.0,True,<NA>,985754,2024-12-19,2953.267000,OK,HQ,HQ-TAD6801-001,TAD
985755,2025-01-01,1.0,0,TR6801-006,14050201,สีสเปรย์ #14(No.226) ส้ม,1.0,Y,1.0,ก.ป.,...,50.0,True,<NA>,985755,2024-05-17,30.374167,OK,HQ,HQ-TR6801-006,TR
985756,2025-01-01,1.0,0,TR6801-006,13017610,"สายพาน 37.5"" 12.5x975",1.0,Y,1.0,เส้น,...,130.0,True,<NA>,985756,2024-12-24,84.958333,OK,HQ,HQ-TR6801-006,TR
985757,2025-01-01,1.0,0,TR6801-006,13014620,"สายพาน 48"" 12.5x1250 แอร์MTXมีP",1.0,Y,2.0,เส้น,...,360.0,True,<NA>,985757,2024-12-05,107.891250,OK,HQ,HQ-TR6801-006,TR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014526,2025-03-09,1.0,0,TAD6803-169,22010003,น้ำมันเบรคเชลล์ บ. 1LT DOT3 SHELL,1.0,Y,2.0,กป.,...,380.0,True,<NA>,1014526,2025-01-31,161.215000,OK,HQ,HQ-TAD6803-169,TAD
1014527,2025-03-09,1.0,0,TAD6803-169,19050021,"แกนสกรู ไม่หัว NC 5/8-2"" VEE",1.0,Y,10.0,ตัว,...,150.0,True,<NA>,1014527,2024-05-30,4.500000,OK,HQ,HQ-TAD6803-169,TAD
1014972,2025-03-10,1.0,0,TAD6803-206,22010037,สเปรย์ทำความสะอาดระบบเบรค CLEANUP 500ml. BENDIX,1.0,Y,1.0,กป.*,...,170.0,True,<NA>,1014972,2025-01-30,102.053333,OK,HQ,HQ-TAD6803-206,TAD
1014973,2025-03-10,1.0,0,TAD6803-206,22051820,น้ำยาหม้อน้ำ มิตซูบิชิ 5L แท้,1.0,Y,1.0,กป.*,...,390.0,True,<NA>,1014973,2025-02-15,276.500000,OK,HQ,HQ-TAD6803-206,TAD


In [58]:
print(sales_filtered["BILLNO"].to_list())


['TAD6801-001', 'TAD6801-001', 'TR6801-006', 'TR6801-006', 'TR6801-006', 'TR6801-006', 'TR6801-007', 'TR6801-007', 'TR6801-007', 'TR6801-008', 'TR6801-008', 'TR6801-008', 'TR6801-008', 'TR6801-008', 'TR6801-008', 'TR6801-008', 'TR6801-008', 'TR6801-010', 'TR6801-010', 'TR6801-010', 'TR6801-010', 'TR6801-011', 'TR6801-012', 'TR6801-012', 'TR6801-013', 'TR6801-014', 'TR6801-014', 'TR6801-015', 'TR6801-015', 'TR6801-015', 'TAD6801-033', 'TR6801-020', 'TR6801-020', 'TR6801-021', 'TR6801-021', 'TR6801-021', 'TAD6801-042', 'TAD6801-042', 'TR6801-023', 'TAD6801-063', 'TAD6801-063', 'TAD6801-064', 'TAD6801-064', 'TAD6801-064', 'TAD6801-064', 'TAD6801-064', 'TAD6801-064', 'TAD6801-065', 'TAD6801-065', 'TAD6801-065', 'TAD6801-065', 'TAD6801-066', 'TAD6801-066', 'TAD6801-066', 'TR6801-024', 'TAD6801-067', 'TR6801-025', 'TAD6801-067', 'TR6801-033', 'TR6801-033', 'TR6801-033', 'TR6801-033', 'TR6801-033', 'TR6801-034', 'TR6801-034', 'TR6801-034', 'TR6801-034', 'TR6801-035', 'TAD6801-113', 'TAD6801-1